In [4]:
# -------------------------------------------------- #
# - Library Dependencies --------------------------- #
# -------------------------------------------------- #

from tkinter import *
import xml.etree.ElementTree as ET
from datetime import datetime
import numpy as np
import time 
from PIL import Image, ImageTk

# -------------------------------------------------- #
# - Read XML Gesture Function ---------------------- #
# -------------------------------------------------- #

def LOAD_GESTURE(filename):
    from xml.dom import minidom
    xmldoc = minidom.parse(str(filename)+'.xml')
    
    gesture_array = []
    
    for stroke in range(len(xmldoc.getElementsByTagName("Stroke"))):
        items = xmldoc.getElementsByTagName("Stroke")[stroke]
        item_list = items.getElementsByTagName("Point")

        for item in item_list:
            x_pos = item.attributes['X'].value
            y_pos = item.attributes['Y'].value
            gesture_array.append([int(x_pos),int(y_pos),stroke])
    
    return gesture_array

# -------------------------------------------------- #
# - $P Recognizer Functions ------------------------ #
# -------------------------------------------------- #

# - Main Recognizer Function ----------------------- #
def P_DOLLAR_RECOGNIZER(candidate_array,templates_array):
    n = 32
    candi = NORMALIZE(candidate_array,n)
    score = float('inf')
    index = None
    for i in range(len(templates)):
        c_t = templates[i]#NORMALIZE(templates[i],n)
        d = GREEDY_CLOUD_MATCH(candi,c_t,n)
        if score > d:
            index = i
            score = d
            result = templates[i]
    return [result,score,index]


def GREEDY_CLOUD_MATCH(candidate_array,template_array,n):
    import math 
    
    epsilon = 0.5
    step = math.floor((n)**(1-epsilon))
    min_dis = float('inf')
    for i in range(0,n,step):
        d1 = CLOUD_DISTANCE(candidate_array,template_array,n,i)
        d2 = CLOUD_DISTANCE(template_array,candidate_array,n,i)
        min_dis = min(min_dis,d1,d2) 
    return min_dis

def CLOUD_DISTANCE(candidate_array,template_array,n,start):
    if start >= n:
        print('Cannot start outside the number of points')
    else:
        matched = [False]*n
        sum_dis = 0
        i = start
        while True:
            index = None
            min_dis = float('inf')
            for j in range(len(matched)):
                if not matched[j]:
                    candidate_point = np.array((candidate_array[i][0],candidate_array[i][1]))
                    template_point = np.array((template_array[j][0],template_array[j][1]))
                    d = EUCLIDIAN_DISTANCE(candidate_point,template_point)
                    if d < min_dis:
                        min_dis = d
                        index = j
            matched[index] = True
            weight = 1-((i-start+n)%n)/n
            sum_dis = sum_dis+(weight*min_dis)
            i = (i+1)%n
            if i == start:
                break
    return sum_dis 

def NORMALIZE(gesture_array,n):
    points = RESAMPLE(gesture_array,n)
    SCALE(points)
    TRANSLATE_TO_ORIGIN(points,n)
    
    return points

def RESAMPLE(gesture_array,n):
    I = PATH_LENGTH(gesture_array)/(n-1)
    D=0
    new_points = []
    
    i=1
    while True:
        if gesture_array[i][2] == gesture_array[i-1][2]:
            point_1 = np.array((gesture_array[i-1][0],gesture_array[i-1][1]))
            point_2 = np.array((gesture_array[i][0],gesture_array[i][1]))
            d = EUCLIDIAN_DISTANCE(point_1,point_2)
            if(D+d) >= I:
                q_x = gesture_array[i-1][0] + ((I-D)/d)*(gesture_array[i][0]-gesture_array[i-1][0])
                q_y = gesture_array[i-1][1] + ((I-D)/d)*(gesture_array[i][1]-gesture_array[i-1][1])
                new_points.append([q_x,q_y])
                gesture_array.insert(i,[q_x,q_y,gesture_array[i][2]])
                D = 0
            else:
                D = D+d
        i=i+1
        if i== len(gesture_array):
            break
            
    if len(new_points) == (n-1):
        new_points.append([gesture_array[len(gesture_array)-1][0],
                            gesture_array[len(gesture_array)-1][1],
                            gesture_array[len(gesture_array)-1][2]])    
            
    if len(new_points) == (n-2):
        new_points.append([gesture_array[len(gesture_array)-1][0],
                            gesture_array[len(gesture_array)-1][1],
                            gesture_array[len(gesture_array)-1][2]]) 
        
        new_points.append([gesture_array[len(gesture_array)-1][0],
                            gesture_array[len(gesture_array)-1][1],
                            gesture_array[len(gesture_array)-1][2]]) 
        
    return new_points
  
    
def SCALE(gesture_array):
    x_min = float('inf')
    x_max = 0
    y_min = float('inf')
    y_max = 0

    for pi in range(len(gesture_array)):
        x_min = min(x_min,gesture_array[pi][0])
        y_min = min(y_min,gesture_array[pi][1])
        x_max = max(x_max,gesture_array[pi][0])
        y_max = max(y_max,gesture_array[pi][1])
    scale = max(x_max-x_min,y_max-y_min)

    for pi in range(len(gesture_array)):
        gesture_array[pi] = ([(gesture_array[pi][0]-x_min)/scale,(gesture_array[pi][1]-y_min)/scale])
    
    return gesture_array    
    
def TRANSLATE_TO_ORIGIN(gesture_array,n):
    c = [0,0]
    
    for pi in range(len(gesture_array)):
        c = [c[0]+gesture_array[pi][0],c[1]+gesture_array[pi][1]]
    c = [c[0]/n,c[1]/n]

    for pi in range(len(gesture_array)):
        gesture_array[pi] = [gesture_array[pi][0]-c[0],gesture_array[pi][1]-c[1]]

    return gesture_array    
    

def EUCLIDIAN_DISTANCE(point_1,point_2):
    import numpy as np
    d_x = point_2[0]-point_1[0]
    d_y = point_2[1]-point_1[1]
    distance = np.sqrt((d_x*d_x)+(d_y*d_y))
    return distance

def PATH_LENGTH(gesture_array):
    d = 0
    i = 1
    while True:
        if gesture_array[i][2] == gesture_array[i-1][2]:
            point_1 = np.array((gesture_array[i-1][0],gesture_array[i-1][1]))
            point_2 = np.array((gesture_array[i][0],gesture_array[i][1]))
            d = d + EUCLIDIAN_DISTANCE(point_1,point_2)
        i =  i+1
        if i == len(gesture_array):
            break
    return d

# -------------------------------------------------- #
# - Parameter Setting for GUI ---------------------- #
# -------------------------------------------------- #

canvas_width = 1280
canvas_height = 720
image = Image.open('Gestures_Default.PNG')

b1 = "up"
xold, yold = None, None
count = 0
gesture = []
file_number = 0

# - Read user defined files ------------------------ #
filename1 = '41-finger-medium-arrowhead-01'
filename2 = '41-finger-medium-asterisk-01'
filename3 = '41-finger-medium-D-01'
filename4 = '41-finger-medium-exclamation_point-01'
filename5 = '41-finger-medium-five_point_star-01'
filename6 = '41-finger-medium-H-01'
filename7 = '41-finger-medium-half_note-01'
filename8 = '41-finger-medium-I-01'
filename9 = '41-finger-medium-line-01'
filename10 = '41-finger-medium-N-01'
filename11 = '41-finger-medium-null-01'
filename12 = '41-finger-medium-P-01'
filename13 = '41-finger-medium-pitchfork-01'
filename14 = '41-finger-medium-six_point_star-01'
filename15 = '41-finger-medium-T-01'
filename16 = '41-finger-medium-X-01'

files = (filename1,filename2,filename3,filename4,filename5
        ,filename6,filename7,filename8,filename9,filename10
        ,filename11,filename12,filename13,filename14,filename15
        ,filename16)

g1 = NORMALIZE(LOAD_GESTURE(filename1),32)
g2 = NORMALIZE(LOAD_GESTURE(filename2),32)
g3 = NORMALIZE(LOAD_GESTURE(filename3),32)
g4 = NORMALIZE(LOAD_GESTURE(filename4),32)
g5 = NORMALIZE(LOAD_GESTURE(filename5),32)
g6 = NORMALIZE(LOAD_GESTURE(filename6),32)
g7 = NORMALIZE(LOAD_GESTURE(filename7),32)
g8 = NORMALIZE(LOAD_GESTURE(filename8),32)
g9 = NORMALIZE(LOAD_GESTURE(filename9),32)
g10 = NORMALIZE(LOAD_GESTURE(filename10),32)
g11 = NORMALIZE(LOAD_GESTURE(filename11),32)
g12 = NORMALIZE(LOAD_GESTURE(filename12),32)
g13 = NORMALIZE(LOAD_GESTURE(filename13),32)
g14 = NORMALIZE(LOAD_GESTURE(filename14),32)
g15 = NORMALIZE(LOAD_GESTURE(filename15),32)
g16 = NORMALIZE(LOAD_GESTURE(filename16),32)

templates = (g1,g2,g3,g4,g5,g6,g7,g8,g9,g10,g11,g12,g13,g14,g15,g16)

# -------------------------------------------------- #
# - GUI Main Function using Tkinter ---------------- #
# -------------------------------------------------- #

def main():
    global drawing_area, text
    root = Tk()
    root.title('$P Recognizer in Python')
    text = StringVar()
    #text.set()
    drawing_area = Canvas(root, width = canvas_width, height = canvas_height, background = 'white')
    drawing_area.grid(row = 0, column =1)
    frame = Frame(root)
    frame.grid(row = 0, column = 0, sticky = 'n')
    #Button1=Button(frame,text="Record", command = clicked, height = 2).grid(row = 3,column = 1, sticky = "we")
    Button2=Button(frame,text="Recognize", command = recognize, height = 2).grid(row = 3,column = 1, sticky = "we")
    photo = ImageTk.PhotoImage(image)
    Image1=Label(frame, image=photo).grid(row = 4,column = 1, sticky = "we")
    Text1 = Label(frame, textvariable = text ).grid(row = 5,column = 1, sticky = "we")
    drawing_area.bind("<Motion>", motion)
    drawing_area.bind("<ButtonPress-1>", b1down)
    drawing_area.bind("<ButtonRelease-1>", b1up)
    root.mainloop()  
    
# - GUI Function Callbacks ------------------------- #
def b1down(event):
    global b1, count
    count = 1 + count
    b1 = "down"

def b1up(event):
    global b1, xold, yold
    b1 = "up"
    xold = None
    yold = None

def motion(event):
    if b1 == "down":
        global xold, yold
        if xold is not None and yold is not None:
            event.widget.create_line(xold,yold,event.x,event.y,smooth=TRUE, width=7)
            gesture.append([event.x,event.y,(count-1)])      
        xold = event.x
        yold = event.y
      
def clicked():
    global file_number
    f = open('file'+str(file_number)+'.xml', 'a')
    print(gesture)
    for i in range(len(gesture)):
        point = ET.Element("Point", X = str(gesture[i][0]), Y = str(gesture[i][1]), strokeid = str(gesture[i][2]))
        data = ET.tostring(point).decode()
        f.write(data)
        f.write('\n')
    f.close()
    file_number = file_number+1   

def recognize():
    global gesture, text
    st = time.time()
    candidate = gesture
    winner = P_DOLLAR_RECOGNIZER(candidate,templates)
    et = time.time()
    name = files[winner[2]]
    text.set('Gesture Recognized Name: '+name+' \n'+ ' Time of Recognition: ' + str(et-st))
    drawing_area.delete('all')
    gesture = []

        
main()